In [1]:
import yaml, os
import sys   
sys.path.append(os.path.abspath('/ist-project/scads/sit/DebiasNeuro-components/'))


In [2]:
from data import get_all_model_paths
from cma_utils import collect_counterfactuals, trace_counterfactual, geting_counterfactual_paths, get_single_representation, geting_NIE_paths, Classifier, test_mask
from cma import scaling_nie_scores, get_topk
import tqdm
import pickle
import operator
from sklearn.preprocessing import MinMaxScaler, Normalizer, QuantileTransformer, RobustScaler

In [3]:
config_path = "../configs/pcgu_config.yaml"
# config_path = "./configs/experiment_config.yaml"
with open(config_path, "r") as yamlfile:
    config = yaml.load(yamlfile, Loader=yaml.FullLoader)
    print(f'config: {config_path}')

config: ../configs/pcgu_config.yaml


In [4]:
def geting_NIE_paths(config, method_name, mode, seed=None):
    NIE_paths = []
    is_NIE_exist = []
    path = f'../../NIE/{method_name}/'
    path = os.path.join(path, "seed_"+ str(config['seed'] if seed is None else seed ) )
    if not os.path.exists(path): os.mkdir(path) 
    layers = config['layers']  if config['computed_all_layers'] else [config['layer']]
    
    if config['is_averaged_embeddings']:
        if config['computed_all_layers']: 
            NIE_path = os.path.join(path, f'avg_embeddings_{mode[0]}_computed_all_layers_.pickle') 
            NIE_paths.append(NIE_path)
            is_NIE_exist.append(os.path.isfile(NIE_path))
        else:
            for layer in layers:
                # if not isinstance(layer, list): cur_layer = [layer]
                NIE_path = os.path.join(path, f'avg_embeddings_{mode[0]}_layer_{layer}_.pickle') 
                NIE_paths.append(NIE_path)
                is_NIE_exist.append(os.path.isfile(NIE_path))
    else:
        for cur_path in config['counterfactual_paths']:
            # extract infor from current path 
            component = sorted(cur_path.split("_"), key=len)[0]  
            class_name = None
            # NIE_path = os.path.join(path, f'avg_high_level_{layer}_{mode[0]}.pickle') 
            NIE_path = os.path.join(path, f'avg_embeddings_{mode[0]}_layer_{layer}_.pickle') 
            print(f"current path: {NIE_path} , is_exist : {os.path.isfile(cur_path)}")
            NIE_paths.append(NIE_path)
            is_NIE_exist.append(os.path.isfile(cur_path))
    return NIE_paths, is_NIE_exist

In [5]:
LOAD_MODEL_PATH = '../../models/debug_baseline/' 
method_name =  'recent_baseline' 
mode = ["High-overlap"]  if config['treatment'] else  ["Low-overlap"] 
seed = config['seed']

NIE_paths = []
if os.path.exists(LOAD_MODEL_PATH): all_model_paths = get_all_model_paths(LOAD_MODEL_PATH)

NIE_paths, _ = geting_NIE_paths(config, method_name, mode, seed=seed)

In [6]:
NIE_paths 

['../../NIE/recent_baseline/seed_3990/avg_embeddings_High-overlap_computed_all_layers_.pickle']

In [7]:
# select candidates  based on percentage
k = config['k']
# select candidates based on the number of neurons
num_top_neurons = config['num_top_neurons']
top_neurons = {}
num_neurons = None
topk = get_topk(config, k=k, num_top_neurons=num_neurons)
key = list(topk.keys())[0]
# rank for NIE
layers = config['layers'] if config['computed_all_layers'] else config['layer']
# compute average NIE
# ranking_nie = {} if config['compute_all_seeds'] else None
import pandas as pd
scores = {"Neuron_ids": None, "NIE_scores": None }

for cur_path in NIE_paths:
    ranking_nie = {}
    with open(cur_path, 'rb') as handle:
        NIE = pickle.load(handle)
        counter = pickle.load(handle)
        print(f"loading NIE : {cur_path}")

    seed = cur_path.split('/')[4].split('_')[-1]
    do = cur_path.split('/')[-1].split('_')[2]
    for layer in layers:
        for component in NIE[do].keys():
            for neuron_id in NIE[do][component][layer].keys():
                NIE[do][component][layer][neuron_id] = NIE[do][component][layer][neuron_id] / counter[do][component][layer][neuron_id]
                ranking_nie[(f"L-{layer}-" if config['computed_all_layers'] else "") + component + "-" + str(neuron_id)] = NIE[do][component][layer][neuron_id].to('cpu')

    # sort whole layers
    if config['computed_all_layers']:
        all_neurons = dict(sorted(ranking_nie.items(), key=operator.itemgetter(1), reverse=True))
        if not isinstance(topk[key], list): topk[key] = [topk[key]]
        for value in topk[key]:
            num_neurons =  len(list(all_neurons.keys())) * value if key == 'percent' else value
            num_neurons = int(num_neurons)
            print(f"++++++++ Component-Neuron_id: {round(value, 2) if key == 'percent' else num_neurons} neurons :+++++++++")
            cur_neurons = sorted(ranking_nie.items(), key=operator.itemgetter(1), reverse=True)
            cur_neurons = dict(cur_neurons)
            scores["Neuron_ids"] = list(cur_neurons.keys())
            scores["NIE_scores"] = list(cur_neurons.values())
            df = pd.DataFrame(scores)
 

loading NIE : ../../NIE/recent_baseline/seed_3990/avg_embeddings_High-overlap_computed_all_layers_.pickle
++++++++ Component-Neuron_id: 0.01 neurons :+++++++++


In [8]:
df['NIE_scores'] = df['NIE_scores'].apply(lambda row :  float(row))

In [9]:
df.head()

,Neuron_ids,NIE_scores
0,L-11-I-877,0.325554
1,L-2-AO-308,0.166183
2,L-3-I-906,0.062301
3,L-11-Q-128,0.060278
4,L-4-I-384,0.051122


In [10]:
nie_scores = df['NIE_scores'].to_numpy()
scaler = MinMaxScaler()
scaler.fit(df['NIE_scores'].to_numpy().reshape(-1, 1))
df['MinMax_NIE'] = scaler.transform(df['NIE_scores'].to_numpy().reshape(-1, 1))
transformer = Normalizer().fit(df['NIE_scores'].to_numpy().reshape(-1, 1))
df['Norm_NIE'] = transformer.transform(df['NIE_scores'].to_numpy().reshape(-1, 1))
qt = QuantileTransformer(output_distribution='normal',random_state=0)
df['Qt_NIE'] = qt.fit_transform(df['NIE_scores'].to_numpy().reshape(-1, 1))
transformer = RobustScaler().fit(df['NIE_scores'].to_numpy().reshape(-1, 1))
df['Robust_NIE'] = transformer.transform(df['NIE_scores'].to_numpy().reshape(-1, 1))

In [11]:
for col in ['NIE_scores', 'MinMax_NIE', 'Norm_NIE', 'Qt_NIE','Robust_NIE']:
    df[f'm-{col}'] = df[col].apply(lambda row : 1-row)

In [17]:
df.head()

,Neuron_ids,NIE_scores,MinMax_NIE,Norm_NIE,Qt_NIE,Robust_NIE,m-NIE_scores,m-MinMax_NIE,m-Norm_NIE,m-Qt_NIE,m-Robust_NIE
0,L-11-I-877,0.325554,1.000000,1.0,5.199338,3177.584058,0.674446,0.000000,0.0,-4.199338,-3176.584058
1,L-2-AO-308,0.166183,0.710837,1.0,3.285856,1622.028484,0.833817,0.289163,0.0,-2.285856,-1621.028484
2,L-3-I-906,0.062301,0.522354,1.0,3.141742,608.080513,0.937699,0.477646,0.0,-2.141742,-607.080513
3,L-11-Q-128,0.060278,0.518683,1.0,3.139500,588.333030,0.939722,0.481317,0.0,-2.139500,-587.333030
4,L-4-I-384,0.051122,0.502070,1.0,3.129544,498.962117,0.948878,0.497930,0.0,-2.129544,-497.962117


In [18]:
!pwd

/ist-project/scads/sit/DebiasNeuro-components/notebooks


In [21]:
with open(f'nie_table.pickle', 'wb') as handle: 
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'nie_table.pickle', 'rb') as handle: 
    read_df = pickle.load(handle) 

In [22]:
read_df.head()

,Neuron_ids,NIE_scores,MinMax_NIE,Norm_NIE,Qt_NIE,Robust_NIE,m-NIE_scores,m-MinMax_NIE,m-Norm_NIE,m-Qt_NIE,m-Robust_NIE
0,L-11-I-877,0.325554,1.000000,1.0,5.199338,3177.584058,0.674446,0.000000,0.0,-4.199338,-3176.584058
1,L-2-AO-308,0.166183,0.710837,1.0,3.285856,1622.028484,0.833817,0.289163,0.0,-2.285856,-1621.028484
2,L-3-I-906,0.062301,0.522354,1.0,3.141742,608.080513,0.937699,0.477646,0.0,-2.141742,-607.080513
3,L-11-Q-128,0.060278,0.518683,1.0,3.139500,588.333030,0.939722,0.481317,0.0,-2.139500,-587.333030
4,L-4-I-384,0.051122,0.502070,1.0,3.129544,498.962117,0.948878,0.497930,0.0,-2.129544,-497.962117


In [16]:
df.iloc[1000:4000]

,Neuron_ids,NIE_scores,MinMax_NIE,Norm_NIE,Qt_NIE,Robust_NIE,m-NIE_scores,m-MinMax_NIE,m-Norm_NIE,m-Qt_NIE,m-Robust_NIE
1000,L-9-O-743,0.002118,0.413157,1.0,2.254970,20.656910,0.997882,0.586843,0.0,-1.254970,-19.656910
1001,L-11-O-197,0.002116,0.413154,1.0,2.254543,20.639289,0.997884,0.586846,0.0,-1.254543,-19.639289
1002,L-4-AO-135,0.002114,0.413149,1.0,2.253886,20.612145,0.997886,0.586851,0.0,-1.253886,-19.612145
1003,L-10-O-362,0.002113,0.413148,1.0,2.253841,20.610295,0.997887,0.586852,0.0,-1.253841,-19.610295
1004,L-9-O-350,0.002113,0.413148,1.0,2.253789,20.608132,0.997887,0.586852,0.0,-1.253789,-19.608132
...,...,...,...,...,...,...,...,...,...,...,...
3995,L-10-O-642,0.000639,0.410473,1.0,1.662454,6.216279,0.999361,0.589527,0.0,-0.662454,-5.216279
3996,L-7-O-213,0.000639,0.410473,1.0,1.662424,6.215915,0.999361,0.589527,0.0,-0.662424,-5.215915
3997,L-5-O-301,0.000639,0.410472,1.0,1.662309,6.214517,0.999361,0.589528,0.0,-0.662309,-5.214517
3998,L-11-O-691,0.000638,0.410472,1.0,1.662212,6.213344,0.999362,0.589528,0.0,-0.662212,-5.213344


In [13]:
df['m-NIE_scores'].describe()

count    82944.000000
mean         0.999938
std          0.002442
min          0.674446
25%          0.999934
50%          0.999998
75%          1.000036
max          1.225591
Name: m-NIE_scores, dtype: float64

In [14]:
df['NIE**'].describe(), df['NIE**'].iloc[:num_neurons].describe()

KeyError: 'NIE**'

In [ ]:
df['m*'] = df['NIE*'].apply(lambda row : 1-row)
df['m**'] = df['NIE**'].apply(lambda row : 1-row)

In [ ]:
df.head()

In [ ]:
df['m*'][df['m*'] > 0.5][:num_neurons]

In [ ]:
df['m*'][:80].plot.kde()

In [ ]:
df['m**'].plot.kde()

In [ ]:
# topk_df['NIE_scores'] = nie_scores
# topk_df['NIE_scores'].plot.kde(), topk_df['NIE_scores'].describe()